# 循环神经网络 (Recurrent Neural Networks)

## 核心概念

循环神经网络(RNN)是一种专门用于处理序列数据的神经网络架构，通过内部的循环连接来保持对历史信息的记忆。

### 核心特性

1. **时序记忆能力** (Temporal Memory)
   - 具有内部状态(隐藏状态)来保存历史信息
   - 当前输出依赖于历史输入序列
   - 适合处理时间依赖性强的数据

2. **参数共享** (Parameter Sharing)
   - 同一组权重在所有时间步上复用
   - 可以处理任意长度的序列
   - 大幅减少参数数量

3. **序列到序列映射**
   - **多对一**: 序列分类(情感分析)
   - **一对多**: 序列生成(图像描述)
   - **多对多(同步)**: 序列标注(词性标注)
   - **多对多(异步)**: 序列翻译(机器翻译)

### 三种主流RNN架构

| 架构 | 优点 | 缺点 | 适用场景 |
|------|------|------|----------|
| **SimpleRNN** | 简单易懂 | 梯度消失严重，无法学习长期依赖 | 短序列、教学演示 |
| **LSTM** | 能学习长期依赖，性能稳定 | 参数多，训练慢 | 长序列、复杂任务 |
| **GRU** | 参数少于LSTM，训练快 | 性能略低于LSTM | 计算资源受限、实时应用 |

---

## 1. 环境准备与数据生成

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

print(f"TensorFlow版本: {tf.__version__}")
print(f"GPU可用: {tf.config.list_physical_devices('GPU')}")

# 设置随机种子
np.random.seed(42)
tf.random.set_seed(42)

---

## 2. SimpleRNN - 基础循环神经网络

### 原理

SimpleRNN是最基础的RNN结构，每个时间步的隐藏状态由当前输入和上一时刻隐藏状态共同决定：

$$h_t = \tanh(W_{hh} h_{t-1} + W_{xh} x_t + b_h)$$

### 梯度消失问题

由于tanh激活函数的导数范围在(0,1)，经过多次连乘后梯度会指数级衰减，导致：
- 难以学习长期依赖
- 只适合短序列(10-20步)
- 训练不稳定

### 演示任务：序列分类

生成一个简单的二分类任务：判断序列均值是否大于0.5

In [ ]:
def generate_sequence_classification_data(n_samples=5000, seq_length=20, n_features=1):
    """
    生成序列分类数据
    规则：序列均值 > 0.5 → 标签1，否则 → 标签0
    """
    X = np.random.rand(n_samples, seq_length, n_features)
    y = (X.mean(axis=1).flatten() > 0.5).astype(int)
    return X, y

# 生成数据
X_seq, y_seq = generate_sequence_classification_data()
X_train_seq, X_test_seq, y_train_seq, y_test_seq = train_test_split(
    X_seq, y_seq, test_size=0.2, random_state=42, stratify=y_seq
)

print(f"训练集形状: {X_train_seq.shape}")  # (4000, 20, 1)
print(f"序列长度: {X_train_seq.shape[1]}")
print(f"特征维度: {X_train_seq.shape[2]}")
print(f"类别分布: {np.bincount(y_train_seq)}")

In [ ]:
def build_simple_rnn(seq_length, n_features, rnn_units=32):
    """
    构建SimpleRNN分类模型
    
    参数:
        seq_length: 序列长度
        n_features: 每个时间步的特征数
        rnn_units: RNN隐藏单元数
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='SimpleRNN_Classifier')
    
    # SimpleRNN层
    # return_sequences=False: 只返回最后一个时间步的输出
    # 用于序列到单值的映射(序列分类)
    model.add(layers.SimpleRNN(
        rnn_units,
        activation='tanh',
        input_shape=(seq_length, n_features),
        return_sequences=False  # 多对一映射
    ))
    
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# 构建并编译模型
model_simple_rnn = build_simple_rnn(
    seq_length=X_train_seq.shape[1],
    n_features=X_train_seq.shape[2]
)

model_simple_rnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_simple_rnn.summary()

In [ ]:
# 训练模型
history_simple_rnn = model_simple_rnn.fit(
    X_train_seq, y_train_seq,
    batch_size=128,
    epochs=100,  # 测试时用2个epoch
    validation_split=0.2,
    verbose=1
)

# 评估
loss, acc = model_simple_rnn.evaluate(X_test_seq, y_test_seq, verbose=0)
print(f"\nSimpleRNN测试精度: {acc:.4f}")

---

## 3. LSTM - 长短期记忆网络

### 核心机制

LSTM通过引入**门控机制**来解决梯度消失问题，包含三个门：

1. **遗忘门 (Forget Gate)**: 决定丢弃哪些历史信息
   $$f_t = \sigma(W_f [h_{t-1}, x_t] + b_f)$$

2. **输入门 (Input Gate)**: 决定更新哪些信息
   $$i_t = \sigma(W_i [h_{t-1}, x_t] + b_i)$$
   $$\tilde{C}_t = \tanh(W_C [h_{t-1}, x_t] + b_C)$$

3. **输出门 (Output Gate)**: 决定输出哪些信息
   $$o_t = \sigma(W_o [h_{t-1}, x_t] + b_o)$$

**细胞状态更新**:
$$C_t = f_t \odot C_{t-1} + i_t \odot \tilde{C}_t$$
$$h_t = o_t \odot \tanh(C_t)$$

### 优势
- 可以学习长期依赖(100+时间步)
- 梯度流动更稳定
- 性能稳定可靠

### 演示任务：IMDB情感分析

In [ ]:
# 加载IMDB数据集（电影评论情感分类）
max_features = 10000  # 词汇表大小
maxlen = 200  # 序列最大长度

(X_train_imdb, y_train_imdb), (X_test_imdb, y_test_imdb) = imdb.load_data(
    num_words=max_features
)

print(f"训练样本数: {len(X_train_imdb)}")
print(f"测试样本数: {len(X_test_imdb)}")
print(f"原始序列长度示例: {len(X_train_imdb[0])}")

# 填充/截断序列到固定长度
X_train_imdb = sequence.pad_sequences(X_train_imdb, maxlen=maxlen)
X_test_imdb = sequence.pad_sequences(X_test_imdb, maxlen=maxlen)

print(f"填充后形状: {X_train_imdb.shape}")  # (25000, 200)

In [ ]:
def build_lstm_model(max_features, maxlen, lstm_units=64, embedding_dim=128):
    """
    构建LSTM情感分析模型
    
    参数:
        max_features: 词汇表大小
        maxlen: 序列长度
        lstm_units: LSTM隐藏单元数
        embedding_dim: 词嵌入维度
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='LSTM_Sentiment')
    
    # Embedding层：将整数序列转为稠密向量
    model.add(layers.Embedding(
        input_dim=max_features,
        output_dim=embedding_dim,
        input_length=maxlen
    ))
    
    # 单层LSTM
    model.add(layers.LSTM(
        lstm_units,
        return_sequences=False  # 只需要最后的输出
    ))
    
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# 构建并编译LSTM模型
model_lstm = build_lstm_model(max_features, maxlen)

model_lstm.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_lstm.summary()

In [ ]:
# 训练LSTM模型（使用简化参数）
history_lstm = model_lstm.fit(
    X_train_imdb, y_train_imdb,
    batch_size=128,
    epochs=10,  # 测试时用1个epoch，实际训练建议5-10
    validation_split=0.2,
    verbose=1
)

# 评估
loss, acc = model_lstm.evaluate(X_test_imdb, y_test_imdb, verbose=0)
print(f"\nLSTM测试精度: {acc:.4f}")

---

## 4. GRU - 门控循环单元

### 核心机制

GRU是LSTM的简化版本，将遗忘门和输入门合并为**更新门**，去掉了细胞状态：

1. **重置门 (Reset Gate)**: 决定保留多少历史信息
   $$r_t = \sigma(W_r [h_{t-1}, x_t])$$

2. **更新门 (Update Gate)**: 决定更新多少新信息
   $$z_t = \sigma(W_z [h_{t-1}, x_t])$$

**隐藏状态更新**:
$$\tilde{h}_t = \tanh(W_h [r_t \odot h_{t-1}, x_t])$$
$$h_t = (1 - z_t) \odot h_{t-1} + z_t \odot \tilde{h}_t$$

### 优势
- 参数量约为LSTM的75%
- 训练速度更快
- 性能接近LSTM
- 更适合实时应用

In [ ]:
def build_gru_model(max_features, maxlen, gru_units=64, embedding_dim=128):
    """
    构建GRU情感分析模型
    
    参数:
        max_features: 词汇表大小
        maxlen: 序列长度
        gru_units: GRU隐藏单元数
        embedding_dim: 词嵌入维度
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='GRU_Sentiment')
    
    model.add(layers.Embedding(
        input_dim=max_features,
        output_dim=embedding_dim,
        input_length=maxlen
    ))
    
    # 单层GRU
    model.add(layers.GRU(
        gru_units,
        return_sequences=False
    ))
    
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# 构建并编译GRU模型
model_gru = build_gru_model(max_features, maxlen)

model_gru.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_gru.summary()

# 参数量对比
lstm_params = model_lstm.count_params()
gru_params = model_gru.count_params()
print(f"\nLSTM参数量: {lstm_params:,}")
print(f"GRU参数量: {gru_params:,}")
print(f"参数减少: {(1 - gru_params/lstm_params)*100:.1f}%")

In [ ]:
# 训练GRU模型
history_gru = model_gru.fit(
    X_train_imdb, y_train_imdb,
    batch_size=128,
    epochs=10,  # 测试时用1个epoch
    validation_split=0.2,
    verbose=1
)

# 评估
loss, acc = model_gru.evaluate(X_test_imdb, y_test_imdb, verbose=0)
print(f"\nGRU测试精度: {acc:.4f}")

---

## 5. 堆叠RNN - 多层RNN架构

### 关键配置

**重要**: 堆叠多层RNN时，**除了最后一层，前面所有层都必须设置 `return_sequences=True`**

- 前面的层需要返回完整序列供下一层处理
- 最后一层根据任务决定：
  - 序列分类: `return_sequences=False`
  - 序列标注: `return_sequences=True`

### 优势
- 增强模型表达能力
- 学习更抽象的特征
- 提升复杂任务性能

### 注意事项
- 层数过多容易过拟合
- 训练时间显著增加
- 通常2-3层已足够

In [ ]:
def build_stacked_lstm(max_features, maxlen, lstm_units=[64, 32], embedding_dim=128):
    """
    构建多层堆叠LSTM模型
    
    参数:
        max_features: 词汇表大小
        maxlen: 序列长度
        lstm_units: 每层LSTM的单元数列表
        embedding_dim: 词嵌入维度
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Stacked_LSTM')
    
    model.add(layers.Embedding(
        input_dim=max_features,
        output_dim=embedding_dim,
        input_length=maxlen
    ))
    
    # 堆叠多层LSTM
    for i, units in enumerate(lstm_units):
        # 除了最后一层，其他层都要返回完整序列
        return_seq = (i < len(lstm_units) - 1)
        model.add(layers.LSTM(
            units,
            return_sequences=return_seq,
            name=f'lstm_{i+1}'
        ))
        model.add(layers.Dropout(0.3))
    
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# 构建3层堆叠LSTM
model_stacked = build_stacked_lstm(
    max_features, 
    maxlen, 
    lstm_units=[64, 32, 16]
)

model_stacked.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_stacked.summary()

In [ ]:
# 训练堆叠LSTM
history_stacked = model_stacked.fit(
    X_train_imdb, y_train_imdb,
    batch_size=128,
    epochs=10,  # 测试时用1个epoch
    validation_split=0.2,
    verbose=1
)

# 评估
loss, acc = model_stacked.evaluate(X_test_imdb, y_test_imdb, verbose=0)
print(f"\n堆叠LSTM测试精度: {acc:.4f}")

---

## 6. 双向RNN - Bidirectional RNN

### 原理

双向RNN同时从前向后和从后向前处理序列，然后合并两个方向的信息：

- **前向RNN**: $\overrightarrow{h}_t = f(\overrightarrow{h}_{t-1}, x_t)$
- **后向RNN**: $\overleftarrow{h}_t = f(\overleftarrow{h}_{t+1}, x_t)$
- **输出**: $h_t = [\overrightarrow{h}_t; \overleftarrow{h}_t]$ (拼接)

### 优势
- 同时利用过去和未来的上下文
- 提升序列理解能力
- 特别适合文本分类、命名实体识别等任务

### 注意事项
- 参数量翻倍
- 不适合实时预测（需要完整序列）
- 适合离线批处理任务

In [ ]:
def build_bidirectional_lstm(max_features, maxlen, lstm_units=64, embedding_dim=128):
    """
    构建双向LSTM模型
    
    参数:
        max_features: 词汇表大小
        maxlen: 序列长度
        lstm_units: LSTM隐藏单元数
        embedding_dim: 词嵌入维度
    
    返回:
        编译后的模型
    """
    model = models.Sequential(name='Bidirectional_LSTM')
    
    model.add(layers.Embedding(
        input_dim=max_features,
        output_dim=embedding_dim,
        input_length=maxlen
    ))
    
    # 双向LSTM
    # 输出维度会翻倍: 2 * lstm_units
    model.add(layers.Bidirectional(
        layers.LSTM(lstm_units, return_sequences=False)
    ))
    
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

In [ ]:
# 构建双向LSTM
model_bidirectional = build_bidirectional_lstm(max_features, maxlen)

model_bidirectional.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model_bidirectional.summary()

In [ ]:
# 训练双向LSTM
history_bidirectional = model_bidirectional.fit(
    X_train_imdb, y_train_imdb,
    batch_size=128,
    epochs=10,  # 测试时用1个epoch
    validation_split=0.2,
    verbose=1
)

# 评估
loss, acc = model_bidirectional.evaluate(X_test_imdb, y_test_imdb, verbose=0)
print(f"\n双向LSTM测试精度: {acc:.4f}")

---

## 7. 时间序列预测示例

演示RNN在时间序列预测中的应用：使用历史数据预测未来值

In [ ]:
def generate_time_series_data(n_samples=1000, seq_length=50):
    """
    生成合成时间序列数据：正弦波 + 噪声
    """
    time = np.arange(n_samples + seq_length + 1)
    series = np.sin(time * 0.1) + np.random.randn(len(time)) * 0.1
    
    # 创建输入输出对
    X, y = [], []
    for i in range(n_samples):
        X.append(series[i:i+seq_length])
        y.append(series[i+seq_length])  # 预测下一个值
    
    return np.array(X).reshape(-1, seq_length, 1), np.array(y)

# 生成数据
X_ts, y_ts = generate_time_series_data()
X_train_ts, X_test_ts, y_train_ts, y_test_ts = train_test_split(
    X_ts, y_ts, test_size=0.2, random_state=42
)

print(f"训练集形状: {X_train_ts.shape}")
print(f"目标形状: {y_train_ts.shape}")

# 可视化部分序列
plt.figure(figsize=(12, 4))
plt.plot(X_train_ts[0].flatten(), label='输入序列')
plt.axvline(x=len(X_train_ts[0])-1, color='r', linestyle='--')
plt.scatter([len(X_train_ts[0])], [y_train_ts[0]], color='r', s=100, label='预测目标')
plt.xlabel('时间步')
plt.ylabel('值')
plt.title('时间序列预测任务示例')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
def build_lstm_forecaster(seq_length, n_features=1, lstm_units=50):
    """
    构建LSTM时间序列预测模型
    """
    model = models.Sequential(name='LSTM_Forecaster')
    
    model.add(layers.LSTM(
        lstm_units,
        return_sequences=True,
        input_shape=(seq_length, n_features)
    ))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.LSTM(lstm_units, return_sequences=False))
    model.add(layers.Dropout(0.2))
    
    model.add(layers.Dense(25, activation='relu'))
    model.add(layers.Dense(1))  # 回归任务，无激活函数
    
    return model

In [ ]:
# 构建并训练预测模型
model_forecaster = build_lstm_forecaster(seq_length=X_train_ts.shape[1])

model_forecaster.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',
    metrics=['mae']
)

history_forecaster = model_forecaster.fit(
    X_train_ts, y_train_ts,
    batch_size=32,
    epochs=100,  # 测试时用2个epoch
    validation_split=0.2,
    verbose=1
)

# 预测并可视化
y_pred_ts = model_forecaster.predict(X_test_ts[:100], verbose=0).flatten()

plt.figure(figsize=(14, 5))
plt.plot(y_test_ts[:100], label='真实值', alpha=0.7)
plt.plot(y_pred_ts, label='预测值', alpha=0.7)
plt.xlabel('样本')
plt.ylabel('值')
plt.title('时间序列预测结果对比')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## 8. 关键知识总结

### RNN架构对比

| 特性 | SimpleRNN | LSTM | GRU |
|------|-----------|------|-----|
| **参数量** | 小 | 大(4倍) | 中(3倍) |
| **训练速度** | 快 | 慢 | 中 |
| **长期依赖** | 差 | 优秀 | 良好 |
| **适用序列长度** | <20 | 100+ | 50+ |
| **门控数量** | 0 | 3 | 2 |
| **推荐场景** | 教学/短序列 | 复杂任务 | 性能/速度平衡 |

### return_sequences参数说明

| 配置 | 输出形状 | 适用场景 |
|------|---------|----------|
| `return_sequences=False` | `(batch, units)` | 序列分类、序列回归 |
| `return_sequences=True` | `(batch, timesteps, units)` | 序列标注、堆叠RNN、Seq2Seq |

**关键规则**:
- 堆叠RNN时，除最后一层外都必须 `return_sequences=True`
- 序列到序列任务(如翻译)，编码器最后一层 `False`，解码器所有层 `True`

### 输入数据格式

RNN要求输入为3D张量: `(batch_size, timesteps, features)`

- `batch_size`: 批量大小
- `timesteps`: 序列长度(时间步数)
- `features`: 每个时间步的特征维度

**示例**:
```python
# 100个样本，每个序列50个时间步，每个时间步1个特征
X = np.random.randn(100, 50, 1)
```

### 常见应用场景

1. **自然语言处理**
   - 情感分析
   - 机器翻译
   - 文本生成
   - 命名实体识别

2. **时间序列分析**
   - 股票价格预测
   - 天气预报
   - 能源负载预测
   - 异常检测

3. **语音处理**
   - 语音识别
   - 语音合成
   - 说话人识别

4. **其他**
   - 视频分析
   - 手势识别
   - 音乐生成

### 优化技巧

1. **梯度裁剪** (Gradient Clipping)
   ```python
   optimizer = keras.optimizers.Adam(clipnorm=1.0)
   ```

2. **学习率调度**
   ```python
   callbacks = [keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)]
   ```

3. **正则化**
   - Dropout (0.2-0.5)
   - Recurrent Dropout
   - 层归一化

4. **批处理**
   - 使用较大batch_size (64-256)
   - 序列填充/截断到固定长度

---

## 完整训练配置模板

```python
# 实际项目中建议使用的完整配置

# 优化器配置
# optimizer = keras.optimizers.Adam(
#     learning_rate=0.001,
#     clipnorm=1.0  # 梯度裁剪
# )

# 回调函数
# callbacks = [
#     keras.callbacks.EarlyStopping(
#         monitor='val_loss',
#         patience=10,
#         restore_best_weights=True
#     ),
#     keras.callbacks.ReduceLROnPlateau(
#         monitor='val_loss',
#         factor=0.5,
#         patience=5,
#         min_lr=1e-7
#     ),
#     keras.callbacks.ModelCheckpoint(
#         'best_rnn_model.h5',
#         monitor='val_accuracy',
#         save_best_only=True
#     )
# ]

# 训练
# history = model.fit(
#     X_train, y_train,
#     batch_size=128,
#     epochs=50,
#     validation_split=0.2,
#     callbacks=callbacks,
#     verbose=1
# )
```

### 性能对比结论

在大多数任务中：
- **首选LSTM**: 复杂任务、长序列、追求最佳性能
- **次选GRU**: 资源受限、实时应用、性能要求适中
- **避免SimpleRNN**: 仅用于教学或极短序列